# Analysis Plots

In [1]:
import sys
sys.path.insert(0, '..')

import pandas as pd
from tgf import TaskGroup, Flag, AbstractFlag, Pipeline
from plotting import PlotPipelineFactory

import warnings
warnings.filterwarnings("ignore")


In [2]:
dataPath  = r"..\data\csv" + "\\"
plotPath = r"..\output\plots\analysis_plots" + "\\"
picklePath = r"..\output\pickle_objects\analysis_pipeline" + "\\"

In [3]:
import pickle
with open(picklePath + 'Analysis Pipeline.pickle', 'rb') as f:
    AnalysisPipeline = pickle.load(f)

AnalysisPipeline.print(priority = True, flags = True)

Analysis Pipeline
	10 Pre Processing: Pre Processing
		10 Select and order columns
		15 Company ID and UUID
			10 Fill Company ID with None
			20 Fill UUID with None
			30 Replace Company IDs
			40 Replace Company UUIDs
		20 MS Data Processing
			10 Fill MS Data with empty String
			20 Length of MS Data
			30 Continuity Type
			40 Drop MS Data Column
		25 Service Data Processing
			10 Fill Service Data with empty String
			20 Length of Service Data
			30 Samsung Type
			40 Drop Service Data Column
		30 Fill Numeric NA with 0
		40 Fill String NA with None
		50 Broadcast
		60 Datetime conversion
		70 Malformed Packet
		80 Order DataFrame
		90 Convert object type to string
	20 Dummy Processing: Dummy Processing
		20 Dummies Protocol
		30 Dummies Channel
		40 Dummies AD Type
		50 Dummies Company
		55 Dummies UUID
		70 Dummies PDU Type
		80 Dummies Continuity Type
		90 Dummies SmartTag Type
	30 Labeling: Labeling
		10 Labeling auto: Labeling auto
			Label AirTag and iPhone: Label AirTag and

In [4]:
AnalysisPipeline_part_1 = AnalysisPipeline[0]
AnalysisPipeline_part_2 = AnalysisPipeline
del AnalysisPipeline_part_2[0]

In [5]:
with open(picklePath +'Pre Processing.pickle', 'rb') as f:
    flag_preProcessing = pickle.load(f)

with open(picklePath +  'Modeling.pickle', 'rb') as f:
    flag_modeling = pickle.load(f)

with open(picklePath + 'Dummy Processing.pickle', 'rb') as f:
    flag_dummy= pickle.load(f)

with open(picklePath + 'Label AirTag.pickle', 'rb') as f:
    flag_label_AirTag = pickle.load(f)

with open(picklePath +  'Label Chipolo.pickle', 'rb') as f:
    flag_label_Chipolo= pickle.load(f)

with open(picklePath + 'Label SkyTag.pickle', 'rb') as f:
    flag_label_SkyTag= pickle.load(f)

with open(picklePath + 'Label SmartTag.pickle', 'rb') as f:
    flag_label_SmartTag = pickle.load(f)

with open(picklePath + 'Label Tile.pickle', 'rb') as f:
    flag_label_Tile = pickle.load(f)

with open(picklePath + 'Label AirTag and iPhone.pickle', 'rb') as f:
    flag_label_AirTag_and_iPhone = pickle.load(f)

with open(picklePath + 'Label Chipolo and iPhone.pickle', 'rb') as f:
    flag_label_Chipolo_and_iPhone = pickle.load(f)

with open(picklePath + 'Label SkyTag and iPhone.pickle', 'rb') as f:
    flag_label_SkyTag_and_iPhone = pickle.load(f)

with open(picklePath + 'Label SmartTag and other Device.pickle', 'rb') as f:
    flag_label_SmartTag_and_Other = pickle.load(f)

with open(picklePath + 'Label Tile and iPhone.pickle', 'rb') as f:
    flag_label_Tile_and_iPhone = pickle.load(f)

with open(picklePath + 'State lost.pickle', 'rb') as f:
    flag_state_lost = pickle.load(f)

with open(picklePath + 'State unpaired.pickle', 'rb') as f:
    flag_state_unpaired = pickle.load(f)

with open(picklePath + 'State nearby.pickle', 'rb') as f:
    flag_state_nearby = pickle.load(f)

with open(picklePath + 'State searching.pickle', 'rb') as f:
    flag_state_searching= pickle.load(f)

with open(picklePath + 'State online.pickle', 'rb') as f:
    flag_state_online= pickle.load(f)

with open(picklePath + 'State offline.pickle', 'rb') as f:
    flag_state_offline= pickle.load(f)

with open(picklePath + 'Label iPhone.pickle', 'rb') as f:
    flag_label_iPhone =  pickle.load(f)

with open(picklePath + 'Label iPad.pickle', 'rb') as f:
    flag_label_iPad=  pickle.load(f)

with open(picklePath + 'Label MacBook.pickle', 'rb') as f:
    flag_label_MacBook =  pickle.load(f)

with open(picklePath + 'Label AirPod.pickle', 'rb') as f:
    flag_label_AirPod = pickle.load(f)

with open(picklePath + 'Label AirPod and iPhone.pickle', 'rb') as f:
    flag_label_AirPod_and_iPhone = pickle.load(f)

with open(picklePath + 'States iDevices.pickle', 'rb') as f:
    flag_states_iDevices = pickle.load(f)

with open(picklePath + 'Label other Device.pickle', 'rb') as f:
    flag_label_other_Device = pickle.load(f)


In [6]:
def getPlotPipeline(titleSuffix: str, labelColumn: str = 'Label', show: bool = False, savePath: str = plotPath, mode:str='analysis') -> TaskGroup:
    return PlotPipelineFactory(titleSuffix=titleSuffix, labelColumn=labelColumn, show=show, savePath=savePath, mode=mode)

In [7]:
def filesToDataFrame(filesDict: dict[str:AbstractFlag], filePath: str = dataPath, labelColumn: str = 'Label', dropLabels:list[str] =None) -> pd.DataFrame:
    if dropLabels is None:
        dropLabels = []
        
    dataset = pd.DataFrame()

    for f in filesDict:
        myPipeline = Pipeline().setPath(filePath + f).loadData()

        new_dataset = myPipeline.setTask(AnalysisPipeline_part_1).run()
        new_dataset_labeled = AnalysisPipeline_part_2.process(new_dataset,flag=filesDict[f])
        
        new_dataset[labelColumn] = new_dataset_labeled[labelColumn]
        new_dataset['File'] = f.split("\\")[-1][:-4]
        dataset = pd.concat([dataset, new_dataset])

        for label in dropLabels:  
            dataset = dataset[~dataset[labelColumn].str.contains(label)]
            
        dataset = dataset[~dataset[labelColumn].isin(dropLabels)]
            
    dataset.reset_index(drop = True, inplace = True)

    return dataset

In [8]:
files = {r"AirTag\AirTag_(nearby).csv": Flag("nearby", parents = [flag_preProcessing, flag_dummy, flag_label_AirTag_and_iPhone, flag_state_nearby]),
         r"AirTag\AirTag_(lost).csv": Flag("lost", parents = [flag_preProcessing, flag_dummy, flag_label_AirTag, flag_state_lost]),
         r"AirTag\AirTag_(unpaired).csv": Flag("unpaired", parents = [flag_preProcessing, flag_dummy, flag_label_AirTag, flag_state_unpaired]),

         r"Chipolo\Chipolo_(nearby).csv": Flag("nearby", parents = [flag_preProcessing, flag_dummy, flag_label_Chipolo_and_iPhone, flag_state_nearby]),
         r"Chipolo\Chipolo_(lost).csv": Flag("lost", parents = [flag_preProcessing, flag_dummy, flag_label_Chipolo, flag_state_lost]),
         r"Chipolo\Chipolo_(unpaired).csv": Flag("unpaired", parents = [flag_preProcessing, flag_dummy, flag_label_Chipolo, flag_state_unpaired]),

         r"SkyTag\SkyTag_(nearby).csv": Flag("nearby", parents = [flag_preProcessing, flag_dummy, flag_label_SkyTag_and_iPhone, flag_state_nearby]),
         r"SkyTag\SkyTag_(lost).csv": Flag("lost", parents = [flag_preProcessing, flag_dummy, flag_label_SkyTag, flag_state_lost]),
         r"SkyTag\SkyTag_(unpaired).csv": Flag("unpaired", parents = [flag_preProcessing, flag_dummy, flag_label_SkyTag, flag_state_unpaired]),
         }

data = filesToDataFrame(files, dropLabels=['iPhone'])

name = 'FindMy Tracker'
plotPipeline = getPlotPipeline(name, savePath=plotPath + name + "\\")
data = plotPipeline.execute(data)

In [9]:
files = {r"SmartTag\SmartTag_(nearby).csv": Flag("nearby", parents = [flag_preProcessing, flag_dummy, flag_label_SmartTag_and_Other, flag_state_nearby]),
         r"SmartTag\SmartTag_(lost).csv": Flag("lost", parents = [flag_preProcessing, flag_dummy, flag_label_SmartTag, flag_state_lost]),
         r"SmartTag\SmartTag_(unpaired).csv": Flag("unpaired", parents = [flag_preProcessing, flag_dummy, flag_label_SmartTag, flag_state_unpaired]),
         r"SmartTag\SmartTag_(searching).csv": Flag("searching", parents = [flag_preProcessing, flag_dummy, flag_label_SmartTag, flag_state_searching])
         }

data = filesToDataFrame(files, dropLabels=['other Device'])

name = 'SmartTag'
plotPipeline = getPlotPipeline(name, savePath=plotPath + name + "\\")
data = plotPipeline.execute(data)

In [10]:
files = {r"Tile\Tile_(nearby).csv": Flag("nearby", parents = [flag_preProcessing, flag_dummy, flag_label_Tile_and_iPhone, flag_state_nearby]),
         r"Tile\Tile_(lost).csv": Flag("lost", parents = [flag_preProcessing, flag_dummy, flag_label_Tile, flag_state_lost]),
         r"Tile\Tile_(unpaired).csv": Flag("unpaired", parents = [flag_preProcessing, flag_dummy, flag_label_Tile, flag_state_unpaired]),
         r"Tile\Tile_(searching).csv": Flag("searching", parents = [flag_preProcessing, flag_dummy, flag_label_Tile, flag_state_searching]),
         }

data = filesToDataFrame(files, dropLabels=['iPhone'])

name = 'Tile'
plotPipeline = getPlotPipeline(name, savePath=plotPath + name + "\\")
data = plotPipeline.execute(data)

In [11]:
files = {r"iPhone\iPhone_(online).csv": Flag("online", parents = [flag_preProcessing, flag_dummy, flag_label_iPhone, flag_state_online]),
         r"iPhone\iPhone_(offline).csv": Flag("offline", parents = [flag_preProcessing, flag_dummy, flag_label_iPhone, flag_state_offline]),
         }

data = filesToDataFrame(files, dropLabels=['iPhone (offline)', 'iPhone (online)'])

name = 'iPhone'
plotPipeline = getPlotPipeline(name, savePath=plotPath + name + "\\")
data = plotPipeline.execute(data)

In [12]:
files = {r"iPad\iPad_(online).csv": Flag("online", parents = [flag_preProcessing, flag_dummy, flag_label_iPad, flag_state_online]),
         r"iPad\iPad_(offline).csv": Flag("offline", parents = [flag_preProcessing, flag_dummy, flag_label_iPad, flag_state_offline]),
         }

data = filesToDataFrame(files, dropLabels=['iPad (offline)', 'iPad (online)', 'iPad CT 09 (offline)'])

name = 'iPad'
plotPipeline = getPlotPipeline(name, savePath=plotPath + name + "\\")
data = plotPipeline.execute(data)

In [13]:
files = {r"MacBook\MacBook_(online).csv": Flag("online", parents = [flag_preProcessing, flag_dummy, flag_label_MacBook, flag_state_online]),
         r"MacBook\MacBook_(offline).csv": Flag("offline", parents = [flag_preProcessing, flag_dummy, flag_label_MacBook, flag_state_offline]),
         }

data = filesToDataFrame(files, dropLabels=['MacBook (offline)', 'MacBook (online)'])

name = 'MacBook'
plotPipeline = getPlotPipeline(name, savePath=plotPath + name + "\\")
data = plotPipeline.execute(data)

In [14]:
files = {r"AirPod\AirPod_(nearby).csv": Flag("nearby", parents = [flag_preProcessing, flag_dummy, flag_label_AirPod_and_iPhone, flag_states_iDevices, flag_state_nearby]),
         r"AirPod\AirPod_(lost).csv": Flag("lost", parents = [flag_preProcessing, flag_dummy, flag_label_AirPod, flag_states_iDevices, flag_state_lost]),
         }

data = filesToDataFrame(files, dropLabels=['AirPod (lost)', 'AirPod (nearby)', 'iPhone', 'AirPod CT 09 (nearby)'])

name = 'AirPod'
plotPipeline = getPlotPipeline(name, savePath=plotPath + name + "\\")
data = plotPipeline.execute(data)

In [15]:
files = {r"other Device\other Device.csv": Flag("other Device", parents = [flag_preProcessing, flag_dummy, flag_label_other_Device]),
         }

data = filesToDataFrame(files)

name = 'other Device'
plotPipeline = getPlotPipeline(name, savePath=plotPath + name + "\\")
data = plotPipeline.execute(data)